In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    IS_COLAB = True
except Exception:
    IS_COLAB = False

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

if not tf.test.is_gpu_available():
    print("No GPU was detected. CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")

# Common imports
import numpy as np
import os

from functools import partial
import numpy as np
from sklearn.datasets import load_sample_image
import tensorflow_datasets as tfds

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "cnn"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

In [2]:
def plot_image(image):
    plt.imshow(image, cmap="gray", interpolation="nearest")
    plt.axis("off")

def plot_color_image(image):
    plt.imshow(image, interpolation="nearest")
    plt.axis("off")

So it looks like we're going to use `Xception` model. Let's start.

## transfer learning

### preprocess data

In [4]:
test_split, valid_split, train_split = tfds.Split.TRAIN.subsplit([10, 15, 75])

test_set_raw = tfds.load("tf_flowers", split=test_split, as_supervised=True)
valid_set_raw = tfds.load("tf_flowers", split=valid_split, as_supervised=True)
train_set_raw = tfds.load("tf_flowers", split=train_split, as_supervised=True)

In [5]:
def preprocess(image, label):
    resized_image = tf.image.resize(image, [224, 224])
    final_image = keras.applications.xception.preprocess_input(resized_image)
    return final_image, label

In [6]:
def central_crop(image):
    shape = tf.shape(image)
    min_dim = tf.reduce_min([shape[0], shape[1]])
    top_crop = (shape[0] - min_dim) // 4
    bottom_crop = shape[0] - top_crop
    left_crop = (shape[1] - min_dim) // 4
    right_crop = shape[1] - left_crop
    return image[top_crop:bottom_crop, left_crop:right_crop]

def random_crop(image):
    shape = tf.shape(image)
    min_dim = tf.reduce_min([shape[0], shape[1]]) * 90 // 100
    return tf.image.random_crop(image, [min_dim, min_dim, 3])

def preprocess(image, label, randomize=False):
    if randomize:
        cropped_image = random_crop(image)
        cropped_image = tf.image.random_flip_left_right(cropped_image)
    else:
        cropped_image = central_crop(image)
    resized_image = tf.image.resize(cropped_image, [224, 224])
    final_image = keras.applications.xception.preprocess_input(resized_image)
    return final_image, label

batch_size = 32
train_set = train_set_raw.shuffle(1000).repeat()
train_set = train_set.map(partial(preprocess, randomize=True)).batch(batch_size).prefetch(1)
valid_set = valid_set_raw.map(preprocess).batch(batch_size).prefetch(1)
test_set = test_set_raw.map(preprocess).batch(batch_size).prefetch(1)

### load info

In [7]:
_, info = tfds.load("tf_flowers", as_supervised=True, with_info=True)

In [8]:
n_classes = info.features["label"].num_classes

In [9]:
n_classes

5

In [15]:
dataset_size = info.splits["train"].num_examples

In [16]:
dataset_size

3670

### load model

In [11]:
base_model = keras.applications.xception.Xception(weights="imagenet",
                                                  include_top=False)
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(n_classes, activation="softmax")(avg)
model = keras.models.Model(inputs=base_model.input, outputs=output)

In [12]:
for layer in base_model.layers:
    layer.trainable = False

In [13]:
optimizer = keras.optimizers.SGD(lr=0.2, momentum=0.9, decay=0.01)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])

In [17]:
history = model.fit(train_set,
                    steps_per_epoch=int(0.75 * dataset_size / batch_size),
                    validation_data=valid_set,
                    validation_steps=int(0.15 * dataset_size / batch_size),
                    epochs=5)

Train for 86 steps, validate for 17 steps
Epoch 1/5
86/86 [==============================] - 45s 528ms/step - loss: 0.7531 - accuracy: 0.7682 - val_loss: 1.2715 - val_accuracy: 0.7665
Epoch 2/5
86/86 [==============================] - 16s 183ms/step - loss: 0.3997 - accuracy: 0.8648 - val_loss: 1.1300 - val_accuracy: 0.7629
Epoch 3/5
86/86 [==============================] - 16s 189ms/step - loss: 0.3283 - accuracy: 0.8870 - val_loss: 1.1823 - val_accuracy: 0.7574
Epoch 4/5
86/86 [==============================] - 17s 197ms/step - loss: 0.3057 - accuracy: 0.9026 - val_loss: 1.3345 - val_accuracy: 0.7500
Epoch 5/5
86/86 [==============================] - 17s 198ms/step - loss: 0.2392 - accuracy: 0.9211 - val_loss: 1.1789 - val_accuracy: 0.7482


In [18]:
for layer in base_model.layers:
    layer.trainable = True

optimizer = keras.optimizers.SGD(learning_rate=0.01, momentum=0.9,
                                 nesterov=True, decay=0.001)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
history = model.fit(train_set,
                    steps_per_epoch=int(0.75 * dataset_size / batch_size),
                    validation_data=valid_set,
                    validation_steps=int(0.15 * dataset_size / batch_size),
                    epochs=5)

Train for 86 steps, validate for 17 steps
Epoch 1/5
86/86 [==============================] - 64s 741ms/step - loss: 0.2660 - accuracy: 0.9088 - val_loss: 0.4354 - val_accuracy: 0.9026
Epoch 2/5
86/86 [==============================] - 55s 640ms/step - loss: 0.1095 - accuracy: 0.9622 - val_loss: 0.3137 - val_accuracy: 0.9154
Epoch 3/5
86/86 [==============================] - 55s 641ms/step - loss: 0.0663 - accuracy: 0.9807 - val_loss: 0.2073 - val_accuracy: 0.9283
Epoch 4/5
86/86 [==============================] - 55s 640ms/step - loss: 0.0352 - accuracy: 0.9902 - val_loss: 0.2372 - val_accuracy: 0.9320
Epoch 5/5
86/86 [==============================] - 55s 640ms/step - loss: 0.0232 - accuracy: 0.9924 - val_loss: 0.2219 - val_accuracy: 0.9467


### resnet

Let's try to use resnet. That's not easy - we don't know the correct learning rate and we get really low `val_accuracy`.

#### lr finder

In [66]:
def train_resnet(lr=.2, epochs=2):
    
    tf.keras.backend.clear_session()
    tf.random.set_seed(42)
    
    base_model_resnet = tf.keras.applications.ResNet152V2(weights="imagenet",
                                                              include_top=False)
    avg = tf.keras.layers.GlobalAveragePooling2D()(base_model_resnet.output)
    output = tf.keras.layers.Dense(n_classes, activation="softmax")(avg)
    model_resnet = tf.keras.models.Model(inputs=base_model_resnet.input, outputs=output)
    
    for layer in base_model_resnet.layers:
        layer.trainable = False
    
    optimizer = tf.keras.optimizers.SGD(lr=lr, momentum=0.9, decay=0.01)
#     optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    model_resnet.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
    
    history_resnet = model_resnet50.fit(train_set,
                    steps_per_epoch=int(0.75 * dataset_size / batch_size),
                    validation_data=valid_set,
                    validation_steps=int(0.15 * dataset_size / batch_size),
                    epochs=epochs)

In [64]:
rates = [1, .5, .1, .05, .001]

In [ ]:
for lr in rates:
    print(f'lr={lr}')
    train_resnet(lr=lr, epochs=1)

#### training

In [76]:
tf.keras.backend.clear_session()
tf.random.set_seed(42)
base_model_resnet = tf.keras.applications.ResNet50V2(weights="imagenet",
                                                              include_top=False)
avg = tf.keras.layers.GlobalAveragePooling2D()(base_model_resnet.output)
output = tf.keras.layers.Dense(n_classes, activation="softmax")(avg)
model_resnet = tf.keras.models.Model(inputs=base_model_resnet.input, outputs=output)

In [77]:
for layer in base_model_resnet.layers:
    layer.trainable = False

In [78]:
optimizer = tf.keras.optimizers.SGD(lr=0.01, momentum=0.9, decay=0.01)
# optimizer = tf.keras.optimizers.Adam(learning_rate=.007)
model_resnet50.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])

In [79]:
history_resnet50 = model_resnet50.fit(train_set,
                    steps_per_epoch=int(0.75 * dataset_size / batch_size),
                    validation_data=valid_set,
                    validation_steps=int(0.15 * dataset_size / batch_size),
                    epochs=5)

Train for 86 steps, validate for 17 steps
Epoch 1/5
86/86 [==============================] - 14s 159ms/step - loss: 0.6275 - accuracy: 0.7958 - val_loss: 1.0973 - val_accuracy: 0.7353
Epoch 2/5
86/86 [==============================] - 10s 119ms/step - loss: 0.4045 - accuracy: 0.8743 - val_loss: 1.1144 - val_accuracy: 0.7353
Epoch 3/5
86/86 [==============================] - 11s 123ms/step - loss: 0.3300 - accuracy: 0.8910 - val_loss: 1.6726 - val_accuracy: 0.7040
Epoch 4/5
86/86 [==============================] - 10s 120ms/step - loss: 0.3408 - accuracy: 0.8899 - val_loss: 1.9261 - val_accuracy: 0.6710
Epoch 5/5
86/86 [==============================] - 10s 118ms/step - loss: 0.2881 - accuracy: 0.8993 - val_loss: 1.7320 - val_accuracy: 0.7151


In [80]:
for layer in base_model_resnet.layers:
    layer.trainable = True

optimizer = keras.optimizers.SGD(learning_rate=0.01, momentum=0.9,
                                 nesterov=True, decay=0.001)
# optimizer = tf.keras.optimizers.Adam(learning_rate=.007/10)
model_resnet.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
history = model_resnet.fit(train_set,
                    steps_per_epoch=int(0.75 * dataset_size / batch_size),
                    validation_data=valid_set,
                    validation_steps=int(0.15 * dataset_size / batch_size),
                    epochs=5)

Train for 86 steps, validate for 17 steps
Epoch 1/5
86/86 [==============================] - 34s 397ms/step - loss: 0.7633 - accuracy: 0.7496 - val_loss: 8.9461 - val_accuracy: 0.5018
Epoch 2/5
86/86 [==============================] - 26s 305ms/step - loss: 0.4325 - accuracy: 0.8474 - val_loss: 1.2736 - val_accuracy: 0.6820
Epoch 3/5
86/86 [==============================] - 26s 299ms/step - loss: 0.3757 - accuracy: 0.8663 - val_loss: 1.2992 - val_accuracy: 0.7445
Epoch 4/5
86/86 [==============================] - 26s 298ms/step - loss: 0.3461 - accuracy: 0.8808 - val_loss: 0.9864 - val_accuracy: 0.7941
Epoch 5/5
86/86 [==============================] - 26s 300ms/step - loss: 0.2928 - accuracy: 0.8906 - val_loss: 0.9057 - val_accuracy: 0.7665
